In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/CNN_AD/Kaggle-4ClassesOfImages.zip

Setup

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL

strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
print(tf.__version__)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BTC_SZ = 16 * strategy.num_replicas_in_sync # here we have used one GPU :: strategy.num.num_replicas_in_sync = 1
IMG_SIZE = [176, 208] 
EPOCHS = 80

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/Alzheimer_s Dataset/train",
    validation_split=0.2,
    subset="training",
    shuffle=True,
    seed=600, #random seed for shuffling and transformations.
    image_size=IMG_SIZE,
    batch_size=BTC_SZ,
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/Alzheimer_s Dataset/train",
    validation_split=0.2,
    subset="validation",
    shuffle=True,
    seed=600, #random seed for shuffling and transformations.
    image_size=IMAGE_SIZE,
    batch_size=BTC_SZ,
)

In [ ]:
CLASS_NAMES = ['MildDementia', 'ModerateDementia', 'NonDementia', 'VeryMildDementia']
train_dataset.class_names =  CLASS_NAMES
validation_dataset.class_names = CLASS_NAMES
NUM_CLASSES = len(CLASS_NAMES)

In [ ]:
# Image Visualization 
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_dataset.class_names[labels[i]])
    plt.axis("off")

In [ ]:
# As this data is not balanced we used one_hot_encodings
def one_hot_label(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

train_dataset = train_dataset.map(one_hot_label, num_parallel_calls=AUTOTUNE)
validaion_dataset = validation_dataset.map(one_hot_label, num_parallel_calls=AUTOTUNE)

In [ ]:
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Code to check how many images in a data under each class
IMAGES_ = []
for label in CLASS_NAMES:
    directory_name = "/content/Alzheimer_s Dataset/train/" + label[:-2] + 'ed'
    IMAGES_.append(len([name for name in os.listdir(directory_name)]))

In [ ]:
IMAGES_

In [ ]:
# Separable convolutions consist of first performing a depthwise spatial convolution
def convolution_block(filters_details):
    convolution_block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters_details, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters_details, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ]
    ) 
    return convolution_block

In [ ]:
def dense_block(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    
    return block

In [ ]:
def define_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(*IMAGE_SIZE, 3)),
        
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        convolution_block(32),
        convolution_block(64),
        
        convolution_block(128),
        tf.keras.layers.Dropout(0.2),
  
        convolution_block(256),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

In [ ]:
with strategy.scope():
    model = define_model()

    METRICS = [tf.keras.metrics.AUC(name='auc')]
    
    model.compile(
        optimizer='adam',
        loss=tf.losses.CategoricalCrossentropy(),
        metrics=METRICS
    )

In [ ]:
def exponential_decay(lr0, s):
    def exponential_decay_function(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_function = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_function)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("alzheimer_model.h5",
                                                    save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler],
    epochs=EPOCHS
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(['auc', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/Alzheimer_s Dataset/test",
    image_size=IMG_SIZE,
    batch_size=BTC_SZ,
)

test_dataset = test_dataset.map(one_hot_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
Test = model.evaluate(test_dataset)